# Smart Workout



# Install Openpose

In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  


Selecting previously unselected package libgflags2.2.
(Reading database ... 145605 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

In [0]:
!rm -rf images
!mkdir images
import os, json
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image 

#Keypoint Generation

In [0]:
!rm -rf openpose/output1
!rm -rf openpose/output2
!cd openpose && ./build/examples/openpose/openpose.bin --video ../expert.mp4 --write_json ./output1/ --display 0  --render_pose 0
!cd openpose && ./build/examples/openpose/openpose.bin --video ../user.mp4 --write_json ./output2/ --display 0  --render_pose 0


Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 6.715588 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 6.344857 seconds.


#Processing of Keypoints

In [0]:
def findKeyPoints(path):
  path_to_json = path

  json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
  json_files.sort()

  points=[]
  for index, js in enumerate(json_files):
      with open(os.path.join(path_to_json, js)) as json_file:
          
          json_text = json.load(json_file)
          if(len(json_text["people"])>=1):
            points.append(json_text["people"][0]["pose_keypoints_2d"])
  k=0
  newPoints=[]
  for x in points:
    new=[]
    for i in range(0,len(x),3):
      new.append((x[i],x[i+1],x[i+2]))
    newPoints.append(new)     
#   print(newPoints)

  X =[]
  Y = []
  C=[]
  for i in newPoints:
    x=[]
    y=[]
    c=[]
    for j in i:
      x.append(j[0])
      y.append(j[1])
      c.append(j[2])
    X.append(x)
    Y.append(y)
    C.append(c)
  return (newPoints, X, Y, C)

  

#Comparison of Angles

In [0]:
def comparison(dict1, dict2):
  result = {}
  for i in range(0, min(len(dict1),len(dict2))):  
    tuple1 = dict1[i]
    tuple2 = dict2[i] 
    
    angle1,angle2 =  makeListOfAngles(tuple1,tuple2)
    diff = []
    k=0
    while(k<len(angle1) or k<len(angle2)):
      if(angle1[k][1] == False or angle2[k][1] == False):  
        diff.append((0,False))
        k += 1
      else:
        difference=abs(angle1[k][0]-angle2[k][0]);
        if(difference <= 0.1872665):
          diff.append((0,False))
        else:
          diff.append((difference, True))
        k += 1
    result[i] = diff
  return result



def func(tuple2, tuple1, tuple3):
  consider=True;
  if(tuple1[2] == 0 or tuple2[2] == 0 or tuple3[2] == 0):
    consider=False
    return (0, consider)
  m1 = math.atan2(tuple2[1]-tuple1[1],tuple2[0]-tuple1[0])
  m2 = math.atan2(tuple3[1]-tuple1[1],tuple3[0]-tuple1[0])
  return (abs(m2-m1), consider)

  
def makeListOfAngles(tuple1,tuple2):
    angle1 = []
  
    angle1.append(func(tuple1[0],tuple1[15],tuple1[17])) #nose reye rear
    angle1.append(func(tuple1[0],tuple1[16],tuple1[18])) #nose leye lear
    angle1.append(func(tuple1[15],tuple1[0],tuple1[16])) #reye nose leye
    angle1.append(func(tuple1[15],tuple1[0],tuple1[1])) #reye nose chest
    angle1.append(func(tuple1[16],tuple1[0],tuple1[1])) #leye nose chest
    angle1.append(func(tuple1[0],tuple1[1],tuple1[2])) #nose chest rshoulder
    angle1.append(func(tuple1[0],tuple1[1],tuple1[5])) #nose chest lshoulder
    angle1.append(func(tuple1[2],tuple1[1],tuple1[8])) #rshoulder chest abdomen
    angle1.append(func(tuple1[5],tuple1[1],tuple1[8])) #lshoulder chest abdomen
    angle1.append(func(tuple1[1],tuple1[2],tuple1[3])) #chest rshoulder relbow
    angle1.append(func(tuple1[1],tuple1[5],tuple1[6]))#chest lshoulder lelbow
    angle1.append(func(tuple1[2],tuple1[3],tuple1[4]))#rshoulder relbow rarm
    angle1.append(func(tuple1[5],tuple1[6],tuple1[7]))#lshoulder lelbow larm
    angle1.append(func(tuple1[1],tuple1[8],tuple1[9]))#chest abdomen rhip
    angle1.append(func(tuple1[1],tuple1[8],tuple1[12]))#chest abdomen lhip
    angle1.append(func(tuple1[8],tuple1[9],tuple1[10]))#abdomen rhip rknee
    angle1.append(func(tuple1[8],tuple1[12],tuple1[13]))#abdomen lhip lknee
    angle1.append(func(tuple1[9],tuple1[8],tuple1[12]))#rhip abdomen lhip
    angle1.append(func(tuple1[9],tuple1[10],tuple1[11]))#rhip rknee rankle
    angle1.append(func(tuple1[12],tuple1[13],tuple1[14]))#lhip lknee lankle
    angle1.append(func(tuple1[10],tuple1[11],tuple1[24]))#rknee rankle rheel
    angle1.append(func(tuple1[13],tuple1[14],tuple1[21]))#lknee lankle lheel
    angle1.append(func(tuple1[11],tuple1[22],tuple1[23]))#rankle rfrontfoot rtoe
    angle1.append(func(tuple1[14],tuple1[19],tuple1[20]))#lankle lfrontfoot ltoe
    angle1.append(func(tuple1[24],tuple1[11],tuple1[22]))#rheel rankle rfrontfoot
    angle1.append(func(tuple1[21],tuple1[14],tuple1[19]))#lheel lankle lfrontfoot
    angle1.append(func(tuple1[2],tuple1[1],tuple1[5]))#rshoulder chest lshoulder
   
    angle2 = []
   
    angle2.append(func(tuple2[0],tuple2[15],tuple2[17])) #nose reye rear
    angle2.append(func(tuple2[0],tuple2[16],tuple2[18])) #nose leye lear
    angle2.append(func(tuple2[15],tuple2[0],tuple2[16])) #reye nose leye
    angle2.append(func(tuple2[15],tuple2[0],tuple2[1])) #reye nose chest
    angle2.append(func(tuple2[16],tuple2[0],tuple2[1])) #leye nose chest
    angle2.append(func(tuple2[0],tuple2[1],tuple2[2])) #nose chest rshoulder
    angle2.append(func(tuple2[0],tuple2[1],tuple2[5])) #nose chest lshoulder
    angle2.append(func(tuple2[2],tuple2[1],tuple2[8])) #rshoulder chest abdomen
    angle2.append(func(tuple2[5],tuple2[1],tuple2[8])) #lshoulder chest abdomen
    angle2.append(func(tuple2[1],tuple2[2],tuple2[3])) #chest rshoulder relbow
    angle2.append(func(tuple2[1],tuple2[5],tuple2[6]))#chest lshoulder lelbow
    angle2.append(func(tuple2[2],tuple2[3],tuple2[4]))#rshoulder relbow rarm
    angle2.append(func(tuple2[5],tuple2[6],tuple2[7]))#lshoulder lelbow larm
    angle2.append(func(tuple2[1],tuple2[8],tuple2[9]))#chest abdomen rhip
    angle2.append(func(tuple2[1],tuple2[8],tuple2[12]))#chest abdomen lhip
    angle2.append(func(tuple2[8],tuple2[9],tuple2[10]))#abdomen rhip rknee
    angle2.append(func(tuple2[8],tuple2[12],tuple2[13]))#abdomen lhip lknee
    angle2.append(func(tuple2[9],tuple2[8],tuple2[12]))#rhip abdomen lhip
    angle2.append(func(tuple2[9],tuple2[10],tuple2[11]))#rhip rknee rankle
    angle2.append(func(tuple2[12],tuple2[13],tuple2[14]))#lhip lknee lankle
    angle2.append(func(tuple2[10],tuple2[11],tuple2[24]))#rknee rankle rheel
    angle2.append(func(tuple2[13],tuple2[14],tuple2[21]))#lknee lankle lheel[(0, False), (0, False), (0, False), (0, False), (0, False), (0, False), (0.2415202306868005, True), (0, False), (0.29741777647249146, True), (0, False), (0.34634776558756464, True), (0, False), (0, False), (0, False), (0, False), (0.24344605245181516, True), (0, False), (0.20710045738501126, True), (0, False), (0.1975046721975131, True), (0, False), (0, False), (0, False), (0, False), (0, False), (0.23129248543623304, True), (0.3145668793628751, True)]
    angle2.append(func(tuple2[11],tuple2[22],tuple2[23]))#rankle rfrontfoot rtoe
    angle2.append(func(tuple2[14],tuple2[19],tuple2[20]))#lankle lfrontfoot ltoe
    angle2.append(func(tuple2[24],tuple2[11],tuple2[22]))#rheel rankle rfrontfoot
    angle2.append(func(tuple2[21],tuple2[14],tuple2[19]))#lheel lankle lfrontfoot
    angle2.append(func(tuple2[2],tuple2[1],tuple2[5]))#rshoulder chest lshoulder
    
    return (angle1,angle2)


## Comparison of Frames



In [0]:
def compare(frame1,frame2):
  angle_tupple =  makeListOfAngles(frame1, frame2)
  angle1 = angle_tupple[0]
  angle2 = angle_tupple[1]
  k=0
  length=0
  diffsum=0
  while(k<len(angle1) or k<len(angle2)):
    if(angle1[k][1] == False or angle2[k][1] == False):
      k += 1
    else:
      difference=(angle1[k][0]-angle2[k][0]);
      if difference<0:
        diffsum += abs(difference)
      
      length += 1
      k += 1
  if(length!=0):
    return diffsum/length
  return 0

def makeSignArray(frame1,frame5):
  angle_tupple =  makeListOfAngles(frame1, frame5)
  angle1 = angle_tupple[0]
  angle2 = angle_tupple[1]
  k=0
  arr={}
  diffsum=0
  while(k<len(angle1) or k<len(angle2)):
    if(angle1[k][1] == False or angle2[k][1] == False):
      k += 1
    else:
      difference=(angle1[k][0]-angle2[k][0]);
      if difference<0:
        arr[k]="-"
      else:
        arr[k]="-"
      k += 1
  return arr

def compareArr(expertArr,noviceArr):
  count=0
  for i in range(0, min(len(expertArr), len(noviceArr))):
    if(expertArr.get(i) and noviceArr.get(i) and expertArr[i]==noviceArr[i]):
      count+=1
  if count > 4:
    return True
  return False

def findFirstFrame(expert,novice):
  i=0
  temp=0
  minimum = 10000000
  Frame=0
  # expertArr=makeSignArray(expert[0],expert[4])
  for j in range(0,100):
    diff=0
    for i in range(0,40):
      diff += compare(novice[j], expert[i])
      j+=1
    j-=20
   
    if(diff/20 < minimum ): 
      
      minimum = diff/20
      Frame = j

  return Frame

#Transformation Matrix Preparation

In [0]:
def makeMatrix(X,Y,s,t):
  t1=[[1,0,-X[1]],[0,1,-Y[1]],[0,0,1]]
  neckt=[[1,0,t[0]],[0,1,t[1]],[0,0,1]]
  s1=[[s,0,0],[0,s,0],[0,0,1]]
  t2=[[1,0,X[1]],[0,1,Y[1]],[0,0,1]]
  m0= matrixMul(neckt,t1)
  m1= matrixMul(s1,m0)
  m2=matrixMul(t2,m1)
  return m2


def findDist(x1,y1,x2,y2):
  dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
  return dist

def matrixMul(A,B):
  result=[]
  for i in range(len(A)): 
    a=[]
    for j in range(len(B[0])): 
      a.append(0)
    result.append(a)
 
  # print(len(A))

  for i in range(0,len(A)): 
   
    # iterating by coloum by B  
  
    for j in range(0,len(B[0])): 
  
        # iterating by rows of B 
        for k in range(0,len(B)): 
            result[i][j] += A[i][k] * B[k][j] 
    
  return result

def findTorsoFactor(X,Y):
  # print(X)
  d1=findDist(X[1],Y[1],X[9],Y[9])
  d2=findDist(X[1],Y[1],X[12],Y[12])
  return (d1+d2)/2

def transformation(X,Y,scale,translate):
  X1=[]
  Y1=[]
  for i in range(len(X)):
    X1.append(0)
    Y1.append(0)
  m2= makeMatrix(X,Y,scale,translate)
  for i in range(0,len(X)):
    temp=[[X[i]],[Y[i]],[1]]
    result=matrixMul(m2,temp)
    X1[i],Y1[i]=result[0][0],result[1][0]
  
  return (X1,Y1)


#Plotting Points on Image

In [0]:
def plotPointsOnImage(X, Y, C,Flag,val,start,changes):
  mapping={"nose":0,"chest":1,"rshoulder":2,"relbow":3,"rarm":4,"lshoulder":5,"lelbow":6,"larm":7,
    "abdomen":8,"rhip":9,"rknee":10,"rankle":11,"lhip":12,"lknee":13,"lankle":14,
    "reye":15,"leye":16,"rear":17,"lear":18,"lbigtoe":19,"lsmalltoe":20,"lheel":21,"rbigtoe":22,"rsmalltoe":23,"rheel":24,"background":25}
        
  string9="000000000"
  for frame in range(0,val):
    for m in range(0,4):
      temp=string9[:len(string9)-len(str(frame*4+m))]
      image='images/user'+temp+str(frame*4+m)+".jpg"

      img = cv2.imread(image,cv2.IMREAD_COLOR)

      i = [17,15,0,16,0,4,3,2,1,5,6,23,22,11,11,10,9,8,12,13,21,14,19,1]
      j = [15,0,16,18,1,3,2,1,5,6,7,22,11,24,10,9,8,12,13,14,14,19,20,8]

      
      
      for z in range(0,len(i)):
        if(C[start+frame][i[z]]==0 or C[start+frame][j[z]]==0) :
          continue
        if(Flag):
          cv2.line(img,(int(X[start+frame][i[z]]),int(Y[start+frame][i[z]])),(int(X[start+frame][j[z]]),int(Y[start+frame][j[z]])),(0,255,0),7)
          p=0
          cv2.putText(img, "difference in angle between", (50,50+20*p), cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0, 0),2, cv2.LINE_AA)
          p+=2
          cv2.putText(img, changes[start+frame], (50,50+20*p), cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0, 0),2, cv2.LINE_AA)
           
        else :
          cv2.line(img,(int(X[start+frame][i[z]]),int(Y[start+frame][i[z]])),(int(X[start+frame][j[z]]),int(Y[start+frame][j[z]])),(0,0,255),7)
        
        status = cv2.imwrite(image,img)







# PDF Generation- Graph of Angles

In [0]:
import matplotlib.backends.backend_pdf
def getAngles(dict1,dict2):
  list1=[]
  list2=[]
  for i in range(0, min(len(dict1),len(dict2))):  #Assumption-> No. of frames are equal
      angles =  makeListOfAngles(dict1[i],dict2[i])
      list1.append(angles[0])
      list2.append(angles[1])
  return (list1,list2)

def angleGraphs(list1,list2):
  frames=min(len(list1),len(list2))
  name=["reye nose chest","leye nose chest","nose chest rshoulder","nose chest lshoulder","rshoulder chest abdomen","lshoulder chest abdomen","chest rshoulder relbow","chest lshoulder lelbow","rshoulder relbow rarm","lshoulder lelbow larm","chest abdomen rhip","chest abdomen lhip","abdomen rhip rknee","abdomen lhip lknee","rhip abdomen lhip","rhip rknee rankle","lhip lknee lankle","rknee rankle rheel","lknee lankle lheel","rankle rfrontfoot rtoe","lankle lfrontfoot ltoe","rheel rankle rfrontfoot","lheel lankle lfrontfoot","rshoulder chest lshoulder"]
  xaxis=list(range(1, frames+1))
  filename="angleplots.pdf"
  pdf = matplotlib.backends.backend_pdf.PdfPages(filename)
  for i in range(3,25):
    f1=[]
    f2=[]
    for j in range(0,frames):  
      f1.append(list1[j][i][0])
      f2.append(list2[j][i][0])
    print(f1)
    fig=plt.figure()
    plt.clf()
    plt.scatter(xaxis,f1,alpha=0.6,s=10)
    plt.scatter(xaxis,f2,alpha=0.6,s=10)
    plt.title(name[i-3]+" angle")
    plt.xlim(1,frames+1)
    plt.ylim(0,10)
    plt.yscale("linear")
    plt.xlabel("frame number")
    plt.ylabel("angle")
    pdf.savefig(fig)
    plt.clf()
  pdf.close()


    



# Main Function

In [0]:
def main():
  tup1 = findKeyPoints('openpose/output1')
  tup2 = findKeyPoints('openpose/output2')

  item1 = {}
  item2 = {}
  for i in range(0, len(tup1[0])):
    item1[i] = tup1[0][i]

  for i in range(0, len(tup2[0])):
    item2[i] = tup2[0][i]

  
  diff = comparison(item1, item2);

  points_to_bodyParts={0:('nose reye rear',0),1:('nose leye lear',0),2:('reye nose leye',0),3:('reye nose chest',0),4:('leye nose chest',0),5:('nose chest rshoulder',0),6:('nose chest lshoulder',0),7:('rshoulder chest abdomen',1),8:('lshoulder chest abdomen',1),9:('chest rshoulder relbow',1),10:('chest lshoulder lelbow',1),11:('rshoulder relbow rarm',1),12:('lshoulder lelbow larm',1),13:('chest abdomen rhip',1),14:('chest abdomen lhip',1),15:('abdomen rhip rknee',1),16:('abdomen lhip lknee',1),17:('rhip abdomen lhip',1),18:('rhip rknee rankle',0),19:('lhip lknee lankle',0),20:('rknee rankle rheel',0),21:('lknee lankle lheel',0),22:('rankle rfrontfoot rtoe',0),23:'lankle lfrontfoot ltoe',24:('rheel rankle rfrontfoot',0),25:('lheel lankle lfrontfoot',0),26:('rshoulder chest lshoulder',1)}

  changes = {}
  for i in range(0, len(diff),15):
    for k in range(i,min(i+15,len(diff))):
      count={}
      for v in range(0,len(diff[k])):
        count[v]=0
      angleDiffList = []
      for j in range(0, len(diff[k])):
        if(diff[k][j][1] == True ):
          if(points_to_bodyParts[j][1] ==1):
            count[j]+=1
            angleDiffList.append(points_to_bodyParts[j][0])

    max_key = max(count, key=lambda s: count[s])


    for z in range(i,i+15):
      changes[z] = points_to_bodyParts[max_key][0]

  newExpertX=[]
  newExpertY=[]
  t1=findTorsoFactor(tup1[1][0],tup1[2][0])
  t2=findTorsoFactor(tup2[1][0],tup2[2][0])
  sf=t2/t1
  tf=[tup2[1][0][1]-tup1[1][0][1],tup2[2][0][1]-tup1[2][0][1]]
  for i in range(len(tup1[1])):
    
    transX,transY = transformation(tup1[1][i],tup1[2][i],sf,tf)
    newExpertX.append(transX)
    newExpertY.append(transY)


  item3={}
  for i in range(0, len(item1)):
    temp=[]
    for j in range(0,len(item1[i])):
      
      temp.append((newExpertX[i][j],newExpertY[i][j],item1[i][j][2]))
    item3[i]=temp

  frameNo = findFirstFrame(item3,item2)+10

  item1 = {}
  item2 = {}
  for i in range(0, len(tup1[0])):
    item1[i] = tup1[0][i]

  for i in range(frameNo, len(tup2[0])):
    item2[i-frameNo] = tup2[0][i]

  
  angles = getAngles(item1, item2);
  angleGraphs(angles[0],angles[1])

  
  minval=min(len(newExpertX),len(tup2[1])-frameNo)
 
  clipped = cv2.VideoCapture('user.mp4')
  totalframes = clipped.get(cv2.CAP_PROP_FRAME_COUNT)
  string9="000000000"
  print(totalframes)
  for frame in range(0,4*minval):
    temp=string9[:len(string9)-len(str(frame))]
    image="images/user"+temp+str(frame)+".jpg"
    img = Image.open("pic.jpg") 
    img.save(image)

  plotPointsOnImage(newExpertX,newExpertY, tup1[3],0,minval,0,changes)
  plotPointsOnImage(tup2[1],tup2[2], tup2[3],1,minval,frameNo,changes)
  path = r'/content/images/user000000002.jpg'
  


if __name__ == "__main__": 
  main()
  

[4.322499266515045, 4.389440628314822, 4.315296399018467, 4.320548994448556, 4.325005062348172, 4.3272449506444355, 4.275879571141888, 4.396019417748349, 4.275180450040153, 4.396138881571667, 4.395641919897176, 4.398892087984917, 4.399225906520551, 4.076769283571865, 4.188179571209049, 4.064748685177043, 4.028361617071276, 3.9283420697744234, 3.9277226828386986, 4.0426728741847455, 4.09861907175539, 4.329305521641811, 4.32915483146455, 4.3296766700616125, 4.33333740002572, 4.197562620266229, 4.198570396088249, 4.197387152509541, 4.196896361912003, 4.331791310888782, 4.332413844462788, 4.33131987034476, 4.330795058898643, 4.397455568909757, 4.3955850336971505, 4.394102515399599, 4.249047842334766, 4.250029637373192, 4.3937805345685375, 4.27512359373433, 4.273859281920668, 4.278826195053861, 4.356690497041935, 4.223894431275676, 4.220104701277975, 4.348945357737339, 4.353624537828267, 4.319624123336031, 4.315337066020968, 4.230405913506215, 4.234354397195381, 4.239525996742671, 4.3480035

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


176.0


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

# Combining Frames to form Video

In [0]:
expert = cv2.VideoCapture('expert.mp4')
fps_expert = expert.get(cv2.CAP_PROP_FPS)
command = "cd images && ffmpeg -framerate " + str(fps_expert) + " -f image2 -s 1920x1080 -pix_fmt yuv420p -i user%09d.jpg -vcodec libx264 -crf " + str(fps_expert) + " out.mp4"
os.system(command)


NameError: ignored